In [1]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"
saved_mab_model = "checkpoints/mab_model_100.pth"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [37]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation."
# user_input = "I am extremely disappointed with the quality; it broke after just one day."
user_input = "Not a good movie!"

content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

Not a good movie!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

negative<|eot_id|>


In [38]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm, hidden_size=1024)
mab_model.to(device)
print()

/tmp/1104405.1.gpu/ipykernel_2118692/3922162976.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), ll

In [39]:
gen_inputs = tokenizer(gen_output, return_tensors="pt").to(device)
input_ids = gen_inputs['input_ids'][:, :-1]
attention_mask = gen_inputs['attention_mask'][:, :-1]
_, mab_values, _ = mab_model.get_dist_value(input_ids, attention_mask)



In [40]:
print(mab_values)
print(mab_values.shape)

tensor([[ 2.5250e-03,  2.5250e-03,  2.5250e-03, -4.6434e-03, -1.2973e-03,
         -5.6778e-03,  3.2543e-03, -2.2803e-03, -5.9467e-03, -1.2179e-02,
         -5.6198e-03, -5.5860e-03, -4.1892e-03,  1.2905e-03, -5.8659e-03,
         -3.2509e-03,  9.7464e-05, -6.6927e-03, -2.0763e-03, -3.4636e-03,
         -5.7192e-03, -3.5522e-03,  2.0858e-04,  6.0480e-03,  1.7499e-03,
          2.1555e-04,  7.8553e-04, -2.6296e-03,  1.8125e-03, -3.8865e-03,
         -1.1446e-03,  3.5391e-04, -2.6837e-03, -3.9003e-03, -4.5659e-03,
         -3.9548e-03,  2.5390e-03, -1.6235e-04, -2.4851e-03,  3.7894e-03,
          9.1882e-04, -4.9345e-03,  1.8881e-03,  3.0723e-03, -4.1324e-03,
          3.2291e-03,  9.6601e-04, -2.0874e-03,  6.8112e-04, -1.1403e-03,
          5.6005e-04, -2.3614e-03, -5.5725e-03, -1.9717e-03, -4.7594e-03,
          1.9095e-04, -4.1705e-03, -3.6226e-03, -5.0264e-03, -3.0790e-03,
         -4.3435e-03, -5.7269e-04,  1.1778e-03, -2.6246e-03,  1.2559e-03,
         -1.8135e-03,  3.7445e-05, -9.

In [41]:
print(input_ids)
print(input_ids.shape)

tensor([[128000, 128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,
          33025,   2696,     25,   6790,    220,   2366,     18,    198,  15724,
           2696,     25,    220,   1627,  10263,    220,   2366,     19,    271,
           2127,  56956,    279,  27065,    315,    279,   2768,  11914,    323,
           6013,    449,   1193,    832,   3492,     25,    364,  31587,   2965,
            364,  43324,   2965,    477,    364,  60668,   2965,   3196,    389,
            279,   8244,  16630,    323,   7438,    315,    279,  11914,     13,
           3234,    539,   3493,    904,   5217,  16540,     13, 128009, 128006,
          52989, 128007,    271,   2688,    264,   1695,   5818,      0, 128009,
         128006,  78191, 128007,    271,  43324]], device='cuda:0')
torch.Size([1, 86])


In [42]:
def visualize_tokens_with_values(input_ids, mab_values, tokenizer):
    # Decode tokens one by one to preserve alignment
    tokens = []
    for i in range(input_ids.shape[1]):
        token = tokenizer.decode(input_ids[0, i:i+1])
        tokens.append(token)
    
    # Normalize MAB values to [0,1] for color intensity first
    normalized_values = (mab_values[0] - mab_values[0].min()) / (mab_values[0].max() - mab_values[0].min())
    
    # Pad normalized_values with a zero at the end
    padded_normalized_values = torch.cat([normalized_values, torch.zeros(1, device=mab_values.device)], dim=0)
    # Pad original mab_values with the last actual value
    padded_mab_values = torch.cat([mab_values[0], mab_values[0][-1:]], dim=0)
    
    # Generate HTML with colored text and values
    html_output = "<div style='font-family: monospace; line-height: 2; background-color: white; padding: 10px;'>"
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        # Use a gradient from white to green
        intensity = float(value)
        green_color = int(intensity * 200)  # Control the maximum intensity
        html_output += f'<span style="color: black; background-color: rgba(0, {green_color}, 0, 0.3); padding: 0.2em; margin: 0.1em; border-radius: 3px;" title="MAB: {orig_value:.3f}, Norm: {value:.3f}">{token}</span>'
    html_output += "</div>"
    
    # Print the values
    print("Token\tNormalized Value\tOriginal MAB Value")
    print("-" * 50)
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        print(f"{token}\t{value:.3f}\t\t{orig_value:.3f}")
    
    from IPython.display import HTML
    return HTML(html_output)

In [43]:
# Usage:
visualization = visualize_tokens_with_values(input_ids, mab_values, tokenizer)
display(visualization)

Token	Normalized Value	Original MAB Value
--------------------------------------------------
<|begin_of_text|>	0.807		0.003
<|begin_of_text|>	0.807		0.003
<|begin_of_text|>	0.807		0.003
<|start_header_id|>	0.413		-0.005
system	0.597		-0.001
<|end_header_id|>	0.357		-0.006


	0.847		0.003
Cut	0.543		-0.002
ting	0.342		-0.006
 Knowledge	0.000		-0.012
 Date	0.360		-0.006
:	0.362		-0.006
 December	0.438		-0.004
 	0.739		0.001
202	0.346		-0.006
3	0.490		-0.003

	0.674		0.000
Today	0.301		-0.007
 Date	0.554		-0.002
:	0.478		-0.003
 	0.354		-0.006
26	0.473		-0.004
 Jul	0.680		0.000
 	1.000		0.006
202	0.764		0.002
4	0.680		0.000


	0.711		0.001
An	0.524		-0.003
alyze	0.768		0.002
 the	0.455		-0.004
 sentiment	0.605		-0.001
 of	0.688		0.000
 the	0.521		-0.003
 following	0.454		-0.004
 sentence	0.418		-0.005
 and	0.451		-0.004
 respond	0.807		0.003
 with	0.659		-0.000
 only	0.532		-0.002
 one	0.876		0.004
 word	0.719		0.001
:	0.397		-0.005
 '	0.772		0.002
positive	0.837		0.003
,'	0.441		-0.004
 